# Socket application with microk8s on Ubuntu 20.04

### Client


                       BankStatement(s)
                              ↓       
          Order(s) -> AccountsReceivable --> Entry(s) -> GeneralLedger
                              ↑                                ↑
                              ↓                                ↓
                            Store                            Store


### Check whether ar-statefulset, gl-statefullset and cnt-statefullset  pods are running

### If pods are not running use next commands from the terminal
- Install microk8s: sudo snap install microk8s --classic
- Enable microk8sbuild-in registry: microk8s enable registry
- Clone files from GitHub: git clone https://github.com/rbontekoe/AR.git
- Enter downloaded files: cd AR
- Download Julia 1.6.0: curl -O https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz
- Create accounts receivable image: docker build --no-cache -f ar.Dockerfile -t localhost:32000/i_ar:v1.0.11 .
- Copy to local registry: docker push localhost:32000/i_ar:v1.0.11
- Create general ledger image: docker build --no-cache -f gl.Dockerfile -t localhost:32000/i_gl:v1.0.3 .
- Copy to local registry: docker push localhost:32000/i_gl:v1.0.3
- Create counter image: docker build --no-cache -f cnt.Dockerfile -t localhost:32000/i_cnt:v1.0.1 .
- Copy to local registry: docker push localhost:32000/i_cnt:v1.0.1
- microk8s.kubectl apply -f ar-storage.yaml
- microk8s.kubectl apply -f gl-storage.yaml
- microk8s.kubectl apply -f cnt-storage.yaml


In [1]:
;microk8s.kubectl get pods -n socket-ns

NAME                READY   STATUS    RESTARTS   AGE
cnt-statefulset-0   1/1     Running   2          46m
gl-statefulset-0    1/1     Running   0          27m
gl-statefulset-1    1/1     Running   0          27m
ar-statefulset-1    1/1     Running   0          2m53s
ar-statefulset-0    1/1     Running   0          2m3s


#### Remove the Previous Generated Files

- sudo rm /var/data-ar/unpaid-invoices.txt /var/data-ar/paid-invoices.txt /var/data-gl/journal.txt /var/data-gl/generalledger.txt /var/data-cnt/seqnbr.txt

# Test Acount Receivable & GeneralLedger Microservices (microk8s)

###  Load the packages

In [2]:
using Sockets, Serialization, AppliSales, AppliAR, AppliGeneralLedger, DataFrames, Query

### Connect to Accounts Receivable (ar-statefulset)

In [3]:
clientside = connect(ip"192.168.2.40", 30012) # connect to account receivable pod ar-statefulset

TCPSocket(RawFD(51) open, 0 bytes waiting)

### Send the sales orders to Accounts Receivable (ar-statefulset)

In [4]:
sales = AppliSales.process() # create sales orders
serialize(clientside, sales) # send orders to account receivable

### Check whether the file unpaid-invoices.txt has been created

In [5]:
; ls /var/data-ar/unpaid-invoices.txt

/var/data-ar/unpaid-invoices.txt


### Display general ledger account 1300 (gl-statefulset)

In [6]:
r = AppliGeneralLedger.read_from_file("/var/data-gl/generalledger.txt")
#df = DataFrame(r)
df = r |> @filter(_.accountid == 1300) |> DataFrame
df[:, [:accountid, :customerid, :invoice_nbr, :debit, :credit, :descr]]

,accountid,customerid,invoice_nbr,debit,credit,descr
,Int64,String,String,Float64,Float64,String
1,1300,Scrooge Investment Bank,ar1-1001,1210.0,0.0,Learn Smiling
2,1300,Duck City Chronicals,ar1-1002,2420.0,0.0,Learn Smiling
3,1300,Donalds Hardware Store,ar1-1003,1210.0,0.0,Learn Smiling


### Modify bank-kubernetes.csv

Modify bank-kubernetes.csv so that invoice_nbr has the correct prefix (ar0 or ar1) and invoice number.

In [7]:
stms = AppliAR.read_bank_statements("./bank-kubernetes.csv") # retrieve the bankstatements

2-element Vector{BankStatement}:
 BankStatement(Dates.Date("2020-01-15"), "Duck City Chronicals Invoice ar1-1002", "NL93INGB", 2420.0)
 BankStatement(Dates.Date("2020-01-15"), "Donalds Hardware Store Bill ar1-1003", "NL39INGB", 1210.0)

### Process the bank statements

In [8]:
serialize(clientside, stms) # create paid invoices and update general ledger

### Check whether the file paid-invoices.txt has been created

In [9]:
; ls /var/data-ar/paid-invoices.txt

/var/data-ar/paid-invoices.txt


### Check whether payments are booked

In [10]:
r2 = AppliGeneralLedger.read_from_file("/var/data-gl/generalledger.txt")
df = r2 |> @filter(_.accountid == 1300) |> DataFrame
df[:, [:accountid, :customerid, :invoice_nbr, :debit, :credit, :descr]]

,accountid,customerid,invoice_nbr,debit,credit,descr
,Int64,String,String,Float64,Float64,String
1,1300,Scrooge Investment Bank,ar1-1001,1210.0,0.0,Learn Smiling
2,1300,Duck City Chronicals,ar1-1002,2420.0,0.0,Learn Smiling
3,1300,Donalds Hardware Store,ar1-1003,1210.0,0.0,Learn Smiling
4,1300,Duck City Chronicals,ar1-1002,0.0,2420.0,Learn Smiling
5,1300,Donalds Hardware Store,ar1-1003,0.0,1210.0,Learn Smiling


### Display the aging report

In [11]:
r1 = AppliAR.aging("/var/data-ar/unpaid-invoices.txt", "/var/data-ar/paid-invoices.txt")
result = DataFrame(r1)

,id_inv,csm,inv_date,amount,days
,String,String,Dates…,Float64,Dates…
1,ar1-1001,Scrooge Investment Bank,"Date(""2021-08-27"")",1210.0,Day(0)


### ================================================================================

# Socket application with Notebooks

__!!! This will not work anymore. Use the previous example with Microk8s (= Kubernetes).__

Usage:
start the two notebooks with the servers and run the code:
- server-ar.ipynb
- server-gl.ipynb

In [1]:
using Sockets, Serialization, AppliSales, AppliAR, AppliGeneralLedger, DataFrames, Query

In [2]:
client = connect(ip"192.168.2.40", 2014)

TCPSocket(RawFD(51) open, 0 bytes waiting)

In [3]:
sales = AppliSales.process() # create orders
serialize(client, sales) # send orders to account receivable

In [6]:
stms = AppliAR.read_bank_statements("./bank.csv")

2-element Vector{BankStatement}:
 BankStatement(Dates.Date("2020-01-15"), "Duck City Chronicals Invoice A-1002", "NL93INGB", 2420.0)
 BankStatement(Dates.Date("2020-01-15"), "Donalds Hardware Store Bill A-1003", "NL39INGB", 1210.0)

In [ ]:
serialize(client, stms)

### ============================================================================

# Test Counter Pod cnt-statefulset-0

In [22]:
using Sockets, Serialization

In [23]:
client = connect(ip"192.168.2.40", 30014)

TCPSocket(RawFD(55) open, 0 bytes waiting)

In [24]:
function retrieve_invoice_nbr(name::String, ip::IPv4, port::Int64)::Int64
    client = connect(ip, port)

    serialize(client, name)

    return deserialize(client)
end # retrieve_invoice_nbr

retrieve_invoice_nbr (generic function with 1 method)

In [27]:
invoice_nbr = retrieve_invoice_nbr("ABC", ip"192.168.2.40", 30014)

1015

### ============================================================================